In [2]:
import pandas as pd
import os

In [5]:
gold_path = '../temp_training/medallion/gold'
gold_files = os.listdir(gold_path)
gold_file_paths = [os.path.join(gold_path, gold_file) for gold_file in gold_files]
gold_file_paths

['../temp_training/medallion/gold\\gold.csv',
 '../temp_training/medallion/gold\\gold_2023-10-20_08-46-12_L14D.csv',
 '../temp_training/medallion/gold\\gold_body_summary.csv',
 '../temp_training/medallion/gold\\gold_NEWS_API_ML_LABELLED_summary.csv',
 '../temp_training/medallion/gold\\gold_NEWS_API_ML_LABELLED_summary_02.csv',
 '../temp_training/medallion/gold\\gold_NEWS_API_TRAINING.csv']

In [12]:
# in this case, the "features" extracted from the silver dataset is the BODY_SUMMARY that is used for zeroshot/fewshot learning
# also, the column 'RELEVANCE_CLASS' is moved to the end as a good practice.
schema_gold = ['INSERT_DATETIME', 'URI', 'TOPIC', 'TOPIC_URI', 'EVENTURI', 'SOURCE', 'URL', 'TITLE', 'BODY', 'BODY_SUMMARY', 'METADATA', 'ARTICLE_HIERARCHY', 'RELEVANCE_CLASS']

# REARRANGE the df to the column order in the schema
# Check the columns from the source df and the schema, if there is a missing column, create it and fill it with empty string
dataframes = [pd.read_csv(gold_file_path) for gold_file_path in gold_file_paths]
results = []

for df in dataframes:
    data_columns = df.columns
    for column in schema_gold:
        # fill missing columns with 
        if column not in (data_columns):
            df[column] = ''
        # rearrange to the order we want in the schema
    df[schema_gold]
    results.append(df)
        

In [32]:
gold_df_combined = pd.concat(results,)
gold_df_combined.fillna(value='', inplace=True)
gold_df_combined.drop(labels=['Unnamed: 0'],axis='columns',inplace=True)
gold_df_combined.reset_index(drop=True, inplace=True)
gold_df_combined = gold_df_combined[gold_df_combined['RELEVANCE_CLASS'].isin([-1,0,1])]
gold_df_combined.groupby('RELEVANCE_CLASS').size()

RELEVANCE_CLASS
-1     603
 0    1558
 1     855
dtype: int64

In [34]:
gold_df_combined.shape

(3016, 13)

In [35]:
gold_df_combined.groupby(['ARTICLE_HIERARCHY']).size()

ARTICLE_HIERARCHY
                   2313
main_article        643
related_article      60
dtype: int64

In [41]:
# saved in excel to prevent csv parsing errors
gold_df_combined.to_excel('../temp_training/medallion/gold/gold_COMBINED.xlsx')